In [1]:
import pandas as pd
import os
import time
import pickle
from surprise import Reader, Dataset, accuracy, SVD, CoClustering, KNNBasic, NMF
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from surprise.model_selection import cross_validate

In [2]:
def read_ratings(ratings_csv: str, data_dir: str = "/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les évaluations des films.

    :param ratings_csv: Nom du fichier CSV contenant les évaluations.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant les évaluations.
    """
    data = pd.read_csv(os.path.join(data_dir, ratings_csv))
    print("Dataset ratings chargé")
    return data

def read_movies(movies_csv: str, data_dir: str = "/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les informations sur les films.

    :param movies_csv: Nom du fichier CSV contenant les informations sur les films.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant les informations sur les films.
    """
    df = pd.read_csv(os.path.join(data_dir, movies_csv))
    print("Dataset movies chargé")
    return df

def create_user_matrix(ratings: pd.DataFrame, movies: pd.DataFrame) -> pd.DataFrame:
    """
    Crée une matrice utilisateur à partir des évaluations et des informations sur les films.

    :param ratings: DataFrame contenant les évaluations.
    :param movies: DataFrame contenant les informations sur les films.
    :return: DataFrame fusionné et prétraité.
    """
    user_encoder = LabelEncoder()
    movie_encoder = LabelEncoder()
    mlb = MultiLabelBinarizer()

    # Fusionner les évaluations et les informations sur les films
    df = ratings.merge(movies[['movieId', 'genres']], on="movieId", how="left")
    df['userId'] = user_encoder.fit_transform(df['userId'])
    df['movieId'] = movie_encoder.fit_transform(df['movieId'])

    # Traitement des genres
    df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres').str.split('|')),
                               columns=mlb.classes_, index=df.index))
    df = df.drop("(no genres listed)", axis=1, errors='ignore')  # Ignore si la colonne n'existe pas

    print("Dataset fusionnés et prétraités")
    return df

def cross_validation_model(df: pd.DataFrame, model):
    reader = Reader(rating_scale = (0.5, 5))
    data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader)
    algo = model
    print("Début de la cross_validation")
    print(cross_validate(algo, data, cv=5))
    return algo

In [ ]:
list_model = ['SVD()',' KNNBasic()', "NMF()", "CoClustering()"]

ratings = read_ratings('ratings.csv')
movies = read_movies('movies.csv')
df = create_user_matrix(ratings, movies)

print(f'Crossvalidation du modèle {list_model[0]}:')
cross_validation_model(df, model = SVD())

### <ins>Résultat SVD</ins>
Crossvalidation du modèle SVD():
Début de la cross_validation
{'test_rmse': array([0.78572473, 0.78542311, 0.78600234, 0.78597007, 0.78619769]),
 'test_mae': array([0.59798303, 0.59756293, 0.59793583, 0.59795825, 0.59826089]),
 'fit_time': (211.97577786445618, 220.31366205215454, 213.17003798484802, 221.2444794178009, 216.3312282562256),
 'test_time': (44.96142315864563, 76.27327013015747, 74.82933473587036, 50.50605344772339, 75.03782200813293)}

In [3]:
list_model = ['SVD()',' KNNBasic()', "NMF()", "CoClustering()"]

ratings = read_ratings('ratings.csv')
movies = read_movies('movies.csv')
df = create_user_matrix(ratings, movies)

print(f'Crossvalidation du modèle {list_model[2]}:')
cross_validation_model(df, model = NMF())

Dataset ratings chargé
Dataset movies chargé
Dataset fusionnés et prétraités
Crossvalidation du modèle NMF():
Début de la cross_validation
{'test_rmse': array([0.86585246, 0.86464799, 0.86499039, 0.86405908, 0.86524613]), 'test_mae': array([0.66504488, 0.66372093, 0.66410269, 0.66347938, 0.66442664]), 'fit_time': (381.69296860694885, 439.98664903640747, 390.18845987319946, 435.57128047943115, 387.13279914855957), 'test_time': (61.81817841529846, 57.02377271652222, 49.743833780288696, 57.646498918533325, 55.05344581604004)}


In [4]:
list_model = ['SVD()',' KNNBasic()', "NMF()", "CoClustering()"]

ratings = read_ratings('ratings.csv')
movies = read_movies('movies.csv')
df = create_user_matrix(ratings, movies)

print(f'Crossvalidation du modèle {list_model[3]}:')
cross_validation_model(df, model = CoClustering())

Dataset ratings chargé
Dataset movies chargé
Dataset fusionnés et prétraités
Crossvalidation du modèle CoClustering():
Début de la cross_validation
{'test_rmse': array([0.87667434, 0.8772441 , 0.87681049, 0.87666407, 0.8776344 ]), 'test_mae': array([0.68403182, 0.68407388, 0.6839259 , 0.6837806 , 0.68466208]), 'fit_time': (416.615526676178, 459.13755464553833, 433.0122127532959, 465.32795763015747, 437.93276023864746), 'test_time': (31.03119993209839, 97.64636063575745, 86.65363144874573, 101.46662425994873, 62.9147469997406)}


### On constate que le modèle le plus efficace (RMSE et MAE la plus basse est le SVD). Nous allons maintenant rechercher les meilleurs paramètres avec GridSearchCV dans le notebook 3
